In [1]:
file = "2017-06-30_14.45.24 (2n1b1r10p)"
yAxis1 = 'CurrentDriversDelay'
# yAxis2 = 'ActiveDrivers'
yAxis2 = 'Errors'
from_index = 0
extra_axis = True

In [2]:
import pandas as pd

from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

df = pd.read_csv("CSV_Files/"+file+".csv", sep=";", parse_dates=["Time"])

df1 = df.set_index(["PcKey", "Timestamp"])
df1 = df1.sort_index()

list_df = []
dfs_yAxis2 = []
pc_ids = df["PcKey"].unique()        # give me all the PcKeys without duplicates
pc_ids = pc_ids[from_index:]

# for each pc
for pc_id in pc_ids:
    df2 = df1.loc[pc_id, ['Time', yAxis1]]    # build a dataframe per vehicle
    df2 = df2.set_index("Time")                         # index by time
    df2 = df2.resample('1S').mean()                    # add rows with 1s difference
    df2 = df2.round(0)
    
    pc_info = {"name": pc_id, "delay": df2[yAxis1]}     # build a dictionary
    list_df.append(pc_info)
    
    if extra_axis:
        df3 = df1.loc[pc_id, ['Time', yAxis2]]    # build a dataframe per vehicle
        df3 = df3.set_index("Time")               # index by time
        df3 = df3.resample('1S').mean()
        df3 = df3.round(0)
        df3 = df3.bfill()
        dfs_yAxis2.append(df3)

<script>requirejs.config({paths: { 'plotly': ['https://cdn.plot.ly/plotly-latest.min']},});if(!window.Plotly) {{require(['plotly'],function(plotly) {window.Plotly=plotly;});}}</script>

In [3]:
aux = len(dfs_yAxis2)-1
df_total = dfs_yAxis2[aux]
while aux >= 0:
    df_total = (df_total.reindex_like(dfs_yAxis2[aux]).fillna(0) + dfs_yAxis2[aux].fillna(0))\
        .fillna(0)
    aux -= 1
    
df_total = {"name": yAxis2, "drivers": df_total[yAxis2]}

In [4]:
import plotly.graph_objs as go
import numpy as np
date = []
traces = []
menu = []
index = 0

for pc_data in list_df:
    traces.append(go.Scatter(x=pc_data["delay"].index, y=pc_data["delay"], name=pc_data["name"],
                             mode='lines+markers', connectgaps=True, opacity=0.8))

if extra_axis:
    traces.append(go.Scatter(x=df_total["drivers"].index, y=df_total["drivers"], 
                             name=df_total["name"], mode='lines+markers', connectgaps=True,
                             opacity=0.8, yaxis='y2'))

menu.append(dict(args=['visible', [True]*(len(list_df))], label='All', method='restyle'))
zeros = np.zeros(len(list_df))

for index, pc_id in enumerate(list_df):
    
    array = [False]*(len(list_df))
    array[index] = True
    menu.append(dict(args=['visible', array], label=pc_id["name"], method='restyle'))


title = "Evolución temporal, file: " + file
updateMenus = list([dict(
        x=-0.15,
        y=1,
        yanchor='top',
        buttons=list(menu),)])

if extra_axis:
    xAxis = dict(
        rangeselector=dict(
            buttons=list([
                dict(count=20,
                     label='20s',
                     step='second',
                     stepmode='backward'),
                dict(count=1,
                     label='1m',
                     step='minute',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=False),
        type='date'
    )
    yAxis = dict(title=yAxis1)
    yAxis2 = dict(
        title=yAxis2,
        titlefont=dict(color='rgb(148, 103, 189)'),
        tickfont=dict(color='rgb(148, 103, 189)'),
        overlaying='y',
        side='right'
    )
    
    layout = Layout(title=title, updatemenus=updateMenus, xaxis=xAxis,
                    yaxis=yAxis, yaxis2=yAxis2, autosize=True)
else:
    layout = Layout(title=title, updatemenus=updateMenus, autosize=True)

fig = Figure(data=traces, layout=layout)
# iplot(fig)    # Looks good, doesnt work xD
plot(fig)

'file://C:\\Users\\Equipo2\\GIT\\CSVPlotter\\temp-plot.html'